# 03 - Build Foundation Sets

## Overview
Join laps with weather and events to create processed base tables.

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

import pandas as pd
from f1ts import config, io_flat, foundation, validation

## Load

In [ ]:
interim_dir = config.paths()['data_interim']
raw_dir = config.paths()['data_raw']

laps_interim = io_flat.read_parquet(interim_dir / 'laps_interim.parquet')
sessions = io_flat.read_csv(raw_dir / 'sessions.csv')

# Load all weather files
weather_files = list(raw_dir.glob('*_weather.csv'))
weather_data = []
for wf in weather_files:
    weather_data.append(pd.read_csv(wf))
weather_raw = pd.concat(weather_data, ignore_index=True) if weather_data else pd.DataFrame()

## Transform

In [ ]:
laps_processed, stints, events = foundation.foundation_pipeline(
    laps_interim, weather_raw, sessions
)

## Validate

In [ ]:
validation.validate_uniqueness(laps_processed, ['session_key', 'driver', 'lap'], 'laps_processed')
print('✓ Validations passed')

## Save

In [ ]:
processed_dir = config.paths()['data_processed']
io_flat.write_parquet(laps_processed, processed_dir / 'laps_processed.parquet')
io_flat.write_parquet(stints, processed_dir / 'stints.parquet')
io_flat.write_parquet(events, processed_dir / 'events.parquet')
print('✓ Saved processed data')

## Repro Notes

- Joined laps with weather and events
- Built stint aggregations
- Extracted racing events from track status